## This converts the multi-tile datasets from EMD files to npz (numpy) files that contain the spectrum array, the HAADF (EM) image and the xray_energies.

In [ ]:
# imports
import numpy as np
import hyperspy.api as hs
import os
import matplotlib.pyplot as plt


# set the desired number of frames (a list) and the home directory
frames = [20]
HomePath = "/path/to/directory"   

# HomePath Structure:
# /path/to/directory
#   |-- EMD    (contains the EMD file)
#   |-- NPZ    (initially empty)

EMDPath = os.path.join(HomePath,'EMD')
file_names = os.listdir(EMDPath)
file_names = [name[:-4] for name in file_names if name.endswith('emd')]


In [ ]:
file_name = file_names[0]
print(file_name)
for last_frame in frames:
    # load new file and save to numpy
    file_path = os.path.join(EMDPath,"%s.emd" % file_name)
    s = hs.load(file_path,SI_dtype='uint8',first_frame=1,last_frame=last_frame,sum_frames=True,select_type = None)
    
    # search for the right data
    for i in range(len(s)):
        if '(2048, 2048|4096)' in repr(s[i]):   
            spectrum_idx = i
        elif 'HAADF' in repr(s[i]): 
            haadf_idx = i
    
    haadf = s[haadf_idx].data       
    spectrum = s[spectrum_idx].data   
    xray_energies = s[spectrum_idx].axes_manager.signal_axes[0].axis

    out_path = os.path.join(HomePath,'NPZ','%s_%03dframes.npz' % (file_name,last_frame))
    np.savez_compressed(out_path, haadf=haadf, spectrum=spectrum,xray_energies=xray_energies)
    print("Saved for %02d frames" % last_frame)
    #print(file_path)
    #f,ax = plt.subplots()
    #ax.imshow(haadf,cmap='gray_r')
    #plt.show()
    del haadf, spectrum, s, xray_energies
